In [2]:
import keyboard
import cv2
import pyautogui
import pygetwindow as gw
import time
import classes.Imcap as Imcap
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
#from main import Imcap
%matplotlib inline
# from tensorflow
fps = 5


Секция записи кадров для обучения:

In [3]:
class read:
    pressed = {'w': False, 'a': False, 's': False, 'd': False, 'shift': False}

    def check_buttons():
        for i in read.pressed:
            if keyboard.is_pressed(i):
                read.pressed[i] = True
            else:
                read.pressed[i] = False


class frame:
    def __init__(self, screen, map, speed, buttons):

        self.screen = screen
        self.map = map
        self.speed = speed
        self.buttons = buttons


In [3]:

class Movements:
    '''use move. method to: do some of this things:'''

    def __init__(self):
        self.pressed = {'w': True, 'a': False, 's': False, 'd': False}

    def gas(self):
        if (self.pressed['s']):
            keyboard.release("s")
            self.pressed['s'] = False
        keyboard.press("w")
        self.pressed['w'] = True

    def brake(self):
        if (self.pressed['w']):
            keyboard.release("w")
            self.pressed['w'] = False
        keyboard.press("s")
        self.pressed['s'] = True

    def roll(self):
        if (self.pressed['w']):
            keyboard.release("w")
            self.pressed['w'] = False
        if (self.pressed['s']):
            keyboard.release("s")
            self.pressed['s'] = False

    def left(self):
        if (self.pressed['d']):
            keyboard.release("d")
            self.pressed['d'] = False
        keyboard.press("a")
        self.pressed['a'] = True

    def right(self):
        if (self.pressed['a']):
            keyboard.release("a")
            self.pressed['a'] = False
        keyboard.press("d")
        self.pressed['d'] = True

    def straight(self):
        if (self.pressed['a']):
            keyboard.release("a")
            self.pressed['a'] = False
        if (self.pressed['d']):
            keyboard.release("d")
            self.pressed['d'] = False

    def realise_all(self):
        keyboard.release("w")
        keyboard.release("a")
        keyboard.release("s")
        keyboard.release("d")


move = Movements()


Основная часть записи:

In [10]:
'''Этот кусочек кода делает скрин'''
window_name = "Need for Speed™ Most Wanted"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
window = gw.getWindowsWithTitle(window_name)[0]
data = np.array([])
if window != []:
    try:
        window.activate()
    except:
        window.maximize()
while True:
    # time.sleep(1/15)
    window_rect = (window.left, window.top, window.width, window.height)

    # Breaking the window into segments:
    map_rect, speed_rect = Imcap.get_rects(window)[0:4]

    nfs_map = np.array(pyautogui.screenshot(region=map_rect))
    nfs_speed = np.array(pyautogui.screenshot(region=speed_rect))
    screen = np.array(pyautogui.screenshot(region=window_rect))

    (thresh, nfs_speed) = cv2.threshold(nfs_speed, 50, 255,
                                        cv2.THRESH_BINARY)  # Darker speedometer pixels screening out
    nfs_speed = cv2.cvtColor(nfs_speed, cv2.COLOR_BGR2GRAY)
    screen = np.array(cv2.cvtColor(screen, cv2.COLOR_BGR2RGB))
    width = int(screen.shape[1] / 5)
    height = int(screen.shape[0] / 5)
    dim = (width, height)
    screen = cv2.resize(screen, dim)
    nfs_map = cv2.cvtColor(nfs_map, cv2.COLOR_BGR2RGB)
    frame_map = np.array(nfs_map)
    speed_list = Imcap.get_speed_list(np.array(nfs_speed))
    speed = Imcap.get_speed(speed_list)
    cv2.imshow("screen", screen)
    read.check_buttons()
    if (speed != 888):
        data = np.append(data, frame(screen, nfs_map, speed, read.pressed))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
np.savez_compressed("game_data2", data)


In [4]:
data = np.hstack([np.load("\data\game_data1.npz"), np.load("\data\game_data2.npz"), np.load("data\game_data3.npz"), np.load(
    "data\game_data4.npz"), np.load("data\game_data6.npz"), np.load("data\Corner chase fast.npz")])


In [21]:
np.load("data/game_data1.npz")
